In [1]:
# standard libraries
import numpy as np
import math
from pprint import pprint as pp
from matplotlib.pyplot import *
%matplotlib inline

In [2]:
# user library
from bayesee.detector.spotter import *
from bayesee.imaging.filter import *
from bayesee.imaging.image import *
from bayesee.operation.nb2d import *
from bayesee.operation.dp import *
from bayesee.operation.mathfunc import *

In [3]:
# stimulus parameters
row = col = 100
i_row = i_col = 128
ppd = 120
cpd = 4
cpi_v = row * cpd / ppd

In [4]:
# simulation parameters
n_bin_spat_sim = 5
n_bin_amp_sim = 3
n_bin_background = 10
n_background = n_bin_background * n_bin_amp_sim * n_bin_spat_sim
background_mean = 0
background_std = 0.204

In [5]:
# templates
rc_template = hann_window(row, col, row/2)
rc_template /= nb2dot(rc_template, rc_template)

cosine_template = hann_window(row, col, row/2) * cosine_wave(row, col, (cpi_v,0))
cosine_template /= nb2dot(cosine_template, cosine_template)

plaid_template = hann_window(row, col, row/2) * cosine_wave(row, col, (cpi_v/2, cpi_v/2)) * cosine_wave(row, col, (cpi_v/2, -cpi_v/2))
plaid_template /= nb2dot(plaid_template, plaid_template)

targets = [rc_template, cosine_template, plaid_template]

In [6]:
# backgrounds
one_f_0_noise = np.zeros((i_row, i_col, n_background))
one_f_noise = np.zeros((i_row, i_col, n_background))
one_f_3_noise = np.zeros((i_row, i_col, n_background))

for i in range(n_background):
    one_f_0_noise[:,:,i] = power_noise(i_row, i_col, 0, background_mean, background_std)
    one_f_noise[:,:,i] = power_noise(i_row, i_col, -1, background_mean, background_std)
    one_f_3_noise[:,:,i] = power_noise(i_row, i_col, -3, background_mean, background_std)
    
backgrounds = [one_f_0_noise, one_f_noise, one_f_3_noise]

In [7]:
# window
hann = flat_top_hann_window(row, col, [0.45 * row, 0.5 * row])

In [8]:
# calculate partial masking factors
pmf_all = np.zeros((len(targets), len(backgrounds), n_background))
spat_sim_all = np.zeros((len(targets), len(backgrounds), n_background))
amp_sim_all = np.zeros((len(targets), len(backgrounds), n_background))

for t, target in enumerate(targets):
    for b, background in enumerate(backgrounds):
        for i in range(n_background):
            pmf_all[t,b,i] = partial_masking_factor(cut_center(background[:,:,i],target), target, hann)
            spat_sim_all[t,b,i] = spatial_cosine_similarity(cut_center(background[:,:,i],target), target, hann)
            amp_sim_all[t,b,i] = amplitude_cosine_similarity(cut_center(background[:,:,i],target), target, hann)

In [9]:
# calculate bin edges
def equal_frequency_bins_2d(x, y, bins):
    # x is binned first, y is then binned in the bins of x
    bins_x = np.interp(np.linspace(0, len(x), bins[0] + 1), np.arange(len(x)), np.sort(x))
    bins_y = np.zeros((bins[0], bins[1]+1))
    for i in range(bins[0]):
        b_y = y[(x>=bins_x[i]) & (x<bins_x[i+1])]
        bins_y[i,:] = np.interp(np.linspace(0, len(b_y), bins[1] + 1), np.arange(len(b_y)), np.sort(b_y))
        
    return bins_x, bins_y

bin_edges_amp_sim_all = np.zeros((len(targets), len(backgrounds), n_bin_amp_sim+1))
bin_edges_spat_sim_all = np.zeros((len(targets), len(backgrounds), n_bin_amp_sim, n_bin_spat_sim+1))

for t, target in enumerate(targets):
    for b, background in enumerate(backgrounds):
        bin_edges_amp_sim_all[t,b,:], bin_edges_spat_sim_all[t,b,:,:] = equal_frequency_bins_2d(amp_sim_all[t,b,:], spat_sim_all[t,b,:], [n_bin_amp_sim, n_bin_spat_sim])

In [10]:
# plot function
def plot_pmf_bin2d(bin_edges_spat_sim_all, bin_edges_amp_sim_all, pmf_all, pargs):
    n_targets, n_backgrounds, n_bin_amp_sim, n_bin_spat_sim  = bin_edges_spat_sim_all.shape
    n_bin_spat_sim -= 1
    n_rows, n_cols = pargs['n_rows'], pargs['n_cols']
    fig, axs = subplots(nrows=n_rows, ncols=n_cols, figsize=pargs['figsize'], constrained_layout=True)
    
    pmf_median = np.zeros((n_bin_amp_sim,n_bin_spat_sim))
    pmf_errors = np.zeros((n_bin_amp_sim,2,n_bin_spat_sim))
    
    for t in range(n_targets):
        for b in range(n_backgrounds):
            if n_targets == 1 or n_backgrounds == 1:
                    a_idx = t if n_backgrounds == 1 else b
            else:
                a_idx = t,b
                    
            x_max = np.abs(bin_edges_spat_sim_all[t,b,:,:]).max()
            
            bin_centers_amp_sim = (bin_edges_amp_sim_all[t,b,:-1]+bin_edges_amp_sim_all[t,b,1:])/2
            bin_centers_spat_sim = (bin_edges_spat_sim_all[t,b,:,:-1]+bin_edges_spat_sim_all[t,b,:,1:])/2
            
            for i in range(n_bin_amp_sim):
                for j in range(n_bin_spat_sim):
                    be_as, be_ss = bin_edges_amp_sim_all[t,b,:], bin_edges_spat_sim_all[t,b,:,:]
                    bin_idx = (amp_sim_all[t,b,:] >= be_as[i]) & (amp_sim_all[t,b,:] < be_as[i+1]) & (spat_sim_all[t,b,:] >= be_ss[i,j]) & (spat_sim_all[t,b,:] < be_ss[i,j+1])
                    
                    pmf_median[i,j] = np.median(pmf_all[t,b,bin_idx])
                    pmf_errors[i,0,j] = np.median(pmf_all[t,b,bin_idx]) - np.quantile(pmf_all[t,b,bin_idx], 0.16)
                    pmf_errors[i,1,j] = np.quantile(pmf_all[t,b,bin_idx], 0.84) - np.median(pmf_all[t,b,bin_idx])
                                                                                    
                axs[a_idx].errorbar(bin_centers_spat_sim[i,:], pmf_median[i,:], yerr=pmf_errors[i,:,:], marker=pargs['markers'][i], c=pargs['colors'][i], mfc=pargs['colors'][i], mec=pargs['colors'][i], ecolor=pargs['colors'][i], capsize=pargs['fontsizes'][1], markersize=pargs['fontsizes'][1], label=f'acs:{bin_centers_amp_sim[i]:.2f}', alpha=pargs['alpha'])
            
            axs[a_idx].text(0.1,0.9,pargs['legends'][t][b], transform=axs[a_idx].transAxes, c='k', fontsize=pargs['fontsizes'][2])
            axs[a_idx].legend(loc='upper right', fontsize=pargs['fontsizes'][2])
            axs[a_idx].set_xlim(-x_max, x_max)
            
            axs[a_idx].tick_params(axis='x', which='both', direction='out', length=0, width=0,pad=5, labelsize=pargs['fontsizes'][2], labelbottom=True, labeltop=False, grid_color='k', grid_alpha=1, grid_linewidth=1, grid_linestyle='--')
            axs[a_idx].grid(visible=True, which='minor', axis='x', linestyle='--', linewidth=pargs['linewidth'])
            axs[a_idx].tick_params(axis='y', which='major', direction='out', length=12, width=4, pad=3, labelsize=pargs['fontsizes'][2], left=True, right=True, labelleft=True, labelright=True)
            axs[a_idx].tick_params(axis='y', which='minor', direction='out', length=8, width=4, left=True, right=True, labelleft=False, labelright=False)
    
    fig.text(0.5, -0.075, pargs['x_label'], ha='center', fontsize=pargs['fontsizes'][0])
    fig.text(-0.05, 0.5, pargs['y_label'], va='center', rotation='vertical', fontsize=pargs['fontsizes'][0])
    savefig('pmf_spat_sim_amp_sim_' + pargs['plot_name'] + '.svg', dpi=300, bbox_inches='tight')
    close()


In [12]:
# plot
n_row = len(targets)
n_col = len(backgrounds)
pargs = {'plot_name': 'rc_cosine_plaid', 'figsize': (20, 8), 'n_rows': n_row, 'n_cols': n_col, 'fontsizes': [36, 15, 12], 'x_label': 'Spatial Cosine Similarity', 'y_label': 'Partial Masking Factor', 'legends': [['rc | white', 'rc | 1/f', 'rc | 1/f^3'], ['cosine | white', 'cosine | 1/f', 'cosine | 1/f^3'], ['plaid | white', 'plaid | 1/f', 'plaid | 1/f^3']], 'markers': ['o', 's', 'D', 'p', 'h', '*'], 'colors':['r', 'g', 'b', 'c', 'm', 'y', 'k'], 'markersize': 100, 'linewidth': 2, 'hist_bins':30, 'alpha': 0.7}
plot_pmf_bin2d(bin_edges_spat_sim_all, bin_edges_amp_sim_all, pmf_all, pargs)